In [ ]:
!curl ifconfig.me

35.199.55.132

In [ ]:
%%sh
curl https://packages.microsoft.com/keys/microsoft.asc | apt-key add -
curl https://packages.microsoft.com/config/ubuntu/$(lsb_release -rs)/prod.list > /etc/apt/sources.list.d/mssql-release.list
sudo apt-get update
sudo ACCEPT_EULA=Y apt-get -q -y install msodbcsql18

OK
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://packages.microsoft.com/ubuntu/22.04/prod jammy InRelease [3,632 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,461 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,572 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-upda

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   975  100   975    0     0   6872      0 --:--:-- --:--:-- --:--:--  6914
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    88  100    88    0     0    681      0 --:--:-- --:--:-- --:--:--   687
W: https://packages.microsoft.com/ubuntu/22.04/prod/dists/jammy/InRelease: Key is stored in legacy trusted.gpg keyring (/etc/apt/trusted.gpg), see the DEPRECATION section in apt-key(8) for details.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be

In [ ]:
!sudo apt-get install unixodbc-dev
!pip install pyodbc
!odbcinst -q -d

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unixodbc-dev is already the newest version (2.3.9-5ubuntu0.1).
unixodbc-dev set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 36 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.2/346.2 kB 22.6 MB/s eta 0:00:00
[ODBC Driver 18 for SQL Server]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import pyodbc
import time

In [ ]:
def batch_insert(cursor, query, data, batch_size=50000):
    cursor.fast_executemany = True
    for start in range(0, len(data), batch_size):
        end = start + batch_size
        cursor.executemany(query, data[start:end])

start = time.time()
print("[DEBUG] Starting ETL process...")

# Step 1: Load and combine both CSVs
print("[DEBUG] Loading and combining CSVs...")
df1 = pd.read_csv("/content/drive/MyDrive/cleaned_data_stock_market/nasdaq/cleaned_combined_data.csv")
df2 = pd.read_csv("/content/drive/MyDrive/cleaned_data_stock_market/nyse/cleaned_combined_data_nyse.csv")
df = pd.concat([df1, df2], ignore_index=True)

# # Trim the data to 10,000 rows for testing/performance
# if len(df) > 5000000:
#     df = df.iloc[:5000000].copy()

# Step 2: Clean column names
print("[DEBUG] Cleaning column names...")
df.columns = [col.lower().strip().replace(" ", "_") for col in df.columns]

# Step 3: Parse date column and drop NaT rows
print("[DEBUG] Parsing date column...")
df["date"] = pd.to_datetime(df["date"], dayfirst=True, errors="coerce")
df = df.dropna(subset=["date"])

# Step 4: Create DimDate
print("[DEBUG] Creating DimDate...")
dim_date = df["date"].drop_duplicates().to_frame()
dim_date["date_id"] = dim_date["date"].dt.strftime('%Y%m%d').astype(int)
dim_date["day"] = dim_date["date"].dt.day
dim_date["month"] = dim_date["date"].dt.month
dim_date["month_name"] = dim_date["date"].dt.strftime('%B')
dim_date["quarter"] = dim_date["date"].dt.quarter
dim_date["year"] = dim_date["date"].dt.year
dim_date["weekday"] = dim_date["date"].dt.day_name()
# Keep 'date' column for merging
# dim_date = dim_date[["date_id", "day", "month", "month_name", "quarter", "year", "weekday"]]

# Debug: Check for duplicate date_id values before inserting into DimDate
num_duplicates = dim_date['date_id'].duplicated().sum()
print(f"[DEBUG] Number of duplicate date_id values in dim_date: {num_duplicates}")
print("[DEBUG] First 5 rows of dim_date:")
print(dim_date.head())

# Step 5: Create DimCompany
dim_company = df["company_symbol"].drop_duplicates().to_frame()
dim_company["company_id"] = range(1, len(dim_company) + 1)
dim_company = dim_company.rename(columns={"company_symbol": "ticker"})
dim_company = dim_company[["company_id", "ticker"]]

# Step 6: Merge surrogate keys
df = df.merge(dim_date, left_on="date", right_on="date", how="left")
df = df.merge(dim_company, left_on="company_symbol", right_on="ticker", how="left")

# After merging, drop 'date' column from dim_date before insert
insert_dim_date = dim_date[["date_id", "day", "month", "month_name", "quarter", "year", "weekday"]]

# Step 7: Create FactStockPrice data
fact_df = df[[
    "date_id", "company_id", "open", "high", "low",
    "close", "adjusted_close", "volume"
]].copy()

# Rename for consistency with DB column names
fact_df.rename(columns={
    "open": "open_price",
    "high": "high_price",
    "low": "low_price",
    "close": "close_price"
}, inplace=True)

# Connect to Azure SQL and insert test rows
print("[DEBUG] Connecting to Azure SQL...")
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 18 for SQL Server};'
    'SERVER=dbadt.database.windows.net;'
    'DATABASE=adt-stock-datawarehouse;UID=vradmin;PWD={Password};'
    'Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
)
print("[DEBUG] Connected to Azure SQL.")

with conn.cursor() as cur:
    print("[DEBUG] Inserting into DimDate...")
    batch_insert(
        cur,
        "INSERT INTO DimDate (date_id, day, month, month_name, quarter, year, weekday) VALUES (?, ?, ?, ?, ?, ?, ?)",
        insert_dim_date.values.tolist()
    )
    conn.commit()
    print("[DEBUG] DimDate insert complete.")

    print("[DEBUG] Inserting into DimCompany...")
    batch_insert(
        cur,
        "INSERT INTO DimCompany (company_id, ticker) VALUES (?, ?)",
        dim_company.values.tolist()
    )
    conn.commit()
    print("[DEBUG] DimCompany insert complete.")

    print("[DEBUG] Inserting into FactStockPrice...")
    batch_insert(
        cur,
        """INSERT INTO FactStockPrice (
            date_id, company_id, open_price, high_price, low_price,
            close_price, adjusted_close, volume
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?)""",
        fact_df.values.tolist()
    )
    conn.commit()
    print("[DEBUG] FactStockPrice insert complete.")

conn.close()
end = time.time()
print(f"✅ All tables populated successfully in {end - start:.2f} seconds.")

[DEBUG] Starting ETL process...
[DEBUG] Loading and combining CSVs...
[DEBUG] Cleaning column names...
[DEBUG] Parsing date column...
[DEBUG] Creating DimDate...
[DEBUG] Number of duplicate date_id values in dim_date: 0
[DEBUG] First 5 rows of dim_date:
        date   date_id  day  month month_name  quarter  year    weekday
0 2005-09-27  20050927   27      9  September        3  2005    Tuesday
1 2005-09-28  20050928   28      9  September        3  2005  Wednesday
2 2005-09-29  20050929   29      9  September        3  2005   Thursday
3 2005-09-30  20050930   30      9  September        3  2005     Friday
4 2005-10-03  20051003    3     10    October        4  2005     Monday
[DEBUG] Connecting to Azure SQL...
[DEBUG] Connected to Azure SQL.
[DEBUG] Inserting into DimDate...
[DEBUG] DimDate insert complete.
[DEBUG] Inserting into DimCompany...
[DEBUG] DimCompany insert complete.
[DEBUG] Inserting into FactStockPrice...
[DEBUG] FactStockPrice insert complete.
✅ All tables populated suc

In [ ]:
import pandas as pd
import pyodbc
import time
import traceback

def batch_insert(cursor, query, data, batch_size=100):
    cursor.fast_executemany = True
    for start in range(0, len(data), batch_size):
        end = min(start + batch_size, len(data))
        batch = data[start:end]
        try:
            cursor.executemany(query, batch)
        except Exception as e:
            print(f"[ERROR] Batch insert failed for rows {start}-{end}: {e}")
            traceback.print_exc()
            # Try row-by-row insert for debug
            for i, row in enumerate(batch, start=start):
                try:
                    cursor.execute(query, row)
                except Exception as row_err:
                    print(f"    [ROW ERROR] Row {i}: {row_err} | Data: {row}")
                    traceback.print_exc()

start = time.time()
print("[DEBUG] Starting full News ETL with table creation...")

# Load news data
news_df = pd.read_csv("cleaned_final_df.csv")
news_df.columns = ["source_url", "article", "ticker_symbol", "company_name", "published_at", "sentiment"]
news_df['published_at'] = pd.to_datetime(news_df['published_at'], errors='coerce')
news_df = news_df.dropna(subset=['published_at'])

# Truncate long text columns (source_url only since article will be dropped)
news_df['source_url'] = news_df['source_url'].astype(str).str.slice(0, 2048)

news_df['date_id'] = news_df['published_at'].dt.strftime('%Y%m%d').astype(int)

# Connect to Azure SQL
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 18 for SQL Server};'
    'SERVER=dbadt.database.windows.net;'
    'DATABASE=adt-stock-datawarehouse;UID=vradmin;PWD={Password};'
    'Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;'
)

# 1. Create DimSentiment if it does not exist
with conn.cursor() as cur:
    cur.execute("""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES
            WHERE TABLE_NAME = 'DimSentiment'
        )
        BEGIN
            CREATE TABLE DimSentiment (
                sentiment_id INT PRIMARY KEY,
                sentiment VARCHAR(20) UNIQUE NOT NULL
            )
        END
    """)
    conn.commit()

# 2. Insert unique sentiment labels
sentiments = pd.DataFrame({'sentiment': ['positive', 'negative', 'neutral']})
with conn.cursor() as cur:
    cur.execute("SELECT sentiment FROM DimSentiment")
    existing_sentiments = set(row[0].lower() for row in cur.fetchall())
    new_sentiments = sentiments[~sentiments['sentiment'].isin(existing_sentiments)]
    if not new_sentiments.empty:
        new_sentiments['sentiment_id'] = range(1, len(new_sentiments) + 1)
        batch_insert(
            cur,
            "INSERT INTO DimSentiment (sentiment_id, sentiment) VALUES (?, ?)",
            new_sentiments[['sentiment_id', 'sentiment']].values.tolist()
        )
        conn.commit()

# 3. Update DimCompany to include company_name
news_companies = news_df[['ticker_symbol', 'company_name']].drop_duplicates()

with conn.cursor() as cur:
    cur.execute("SELECT company_id, ticker FROM DimCompany")
    columns = [desc[0] for desc in cur.description]
    rows = cur.fetchall()
    print(f"[DEBUG] Column names in DimCompany: {columns}")
    print(f"[DEBUG] Sample row: {rows[0] if rows else 'No rows found'}")
    existing = pd.DataFrame.from_records(rows, columns=columns)

merged = pd.merge(existing, news_companies, how='left', left_on='ticker', right_on='ticker_symbol')
merged = merged.dropna(subset=["company_name"]).drop_duplicates()

with conn.cursor() as cur:
    for row in merged.itertuples(index=False):
        cur.execute(
            "UPDATE DimCompany SET company_name = ? WHERE company_id = ?",
            row.company_name, row.company_id
        )
    conn.commit()
print(f"[DEBUG] DimCompany updated with {len(merged)} company_name entries.")

# 4. Insert new dates into DimDate
news_dates = news_df['published_at'].dt.normalize().drop_duplicates().to_frame(name='date')
news_dates['date_id'] = news_dates['date'].dt.strftime('%Y%m%d').astype(int)
news_dates['day'] = news_dates['date'].dt.day
news_dates['month'] = news_dates['date'].dt.month
news_dates['month_name'] = news_dates['date'].dt.strftime('%B')
news_dates['quarter'] = news_dates['date'].dt.quarter
news_dates['year'] = news_dates['date'].dt.year
news_dates['weekday'] = news_dates['date'].dt.day_name()

with conn.cursor() as cur:
    cur.execute('SELECT date_id FROM DimDate')
    existing_date_ids = set(row[0] for row in cur.fetchall())
    new_dates = news_dates[~news_dates['date_id'].isin(existing_date_ids)]
    if not new_dates.empty:
        batch_insert(
            cur,
            "INSERT INTO DimDate (date_id, day, month, month_name, quarter, year, weekday) VALUES (?, ?, ?, ?, ?, ?, ?)",
            new_dates[['date_id', 'day', 'month', 'month_name', 'quarter', 'year', 'weekday']].values.tolist()
        )
        conn.commit()

# 5. Create FactNews table if not exists (without article column in fact table)
with conn.cursor() as cur:
    cur.execute("""
        IF NOT EXISTS (
            SELECT * FROM INFORMATION_SCHEMA.TABLES
            WHERE TABLE_NAME = 'FactNews'
        )
        BEGIN
            CREATE TABLE FactNews (
                fact_id INT IDENTITY(1,1) PRIMARY KEY,
                date_id INT,
                company_id INT,
                sentiment_id INT,
                source_url VARCHAR(MAX),
                FOREIGN KEY (date_id) REFERENCES DimDate(date_id),
                FOREIGN KEY (company_id) REFERENCES DimCompany(company_id),
                FOREIGN KEY (sentiment_id) REFERENCES DimSentiment(sentiment_id)
            )
        END
    """)
    conn.commit()

# 6. Prepare FactNews data (drop article column)
with conn.cursor() as cur:
    cur.execute('SELECT ticker, company_id FROM DimCompany')
    company_map = {row[0]: row[1] for row in cur.fetchall()}
    cur.execute('SELECT sentiment, sentiment_id FROM DimSentiment')
    sentiment_map = {row[0].lower(): row[1] for row in cur.fetchall()}

news_df['company_id'] = news_df['ticker_symbol'].map(company_map)
news_df['sentiment_id'] = news_df['sentiment'].str.lower().map(sentiment_map)

fact_news = news_df[['date_id', 'company_id', 'sentiment_id', 'source_url']].copy()
fact_news = fact_news.dropna(subset=['date_id', 'company_id', 'sentiment_id'])

# Convert IDs explicitly to int
fact_news['date_id'] = fact_news['date_id'].astype(int)
fact_news['company_id'] = fact_news['company_id'].astype(int)
fact_news['sentiment_id'] = fact_news['sentiment_id'].astype(int)

print("[DEBUG] fact_news null counts:")
print(fact_news.isnull().sum())
print("[DEBUG] fact_news dtypes:")
print(fact_news.dtypes)
print("[DEBUG] Inserting", len(fact_news), "rows into FactNews...")
print("[DEBUG] Sample row:", fact_news.iloc[0].to_dict())

# 7. Insert into FactNews
with conn.cursor() as cur:
    if not fact_news.empty:
        batch_insert(
            cur,
            "INSERT INTO FactNews (date_id, company_id, sentiment_id, source_url) VALUES (?, ?, ?, ?)",
            fact_news.values.tolist()
        )
        conn.commit()
    else:
        print('[DEBUG] No valid rows to insert into FactNews.')

conn.close()
end = time.time()
print(f"All tables created/updated in {end - start:.2f} seconds.")